# Choosing a kernel for your GP
The critical piece for a Gaussian process is the kernel function that specifies how the covariance matrix looks. You can read all about how to choose them [here](http://www.cs.toronto.edu/~duvenaud/cookbook/index.html). In this notebook I'll actually show you what happens when you choose different kernels for different problems, and what the effects are.